In [1]:
#import dependencies
import pandas as pd
import numpy as np
import json

In [9]:
file_dir="Data/"

In [10]:
with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw=json.load(file)

In [11]:
len(wiki_movies_raw)

7311

In [19]:
kaggle_metadata=pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings=pd.read_csv(f'{file_dir}ratings.csv')

In [24]:
kaggle_metadata[kaggle_metadata["original_language"]=='en'].sample(n=5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
30619,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,219666,tt2338874,en,Laiškai Sofijai,"Based on a true story, this is the love affair...",...,2013-08-28,0.0,0.0,"[{'iso_639_1': 'lt', 'name': 'Lietuvi\x9akai'}...",Released,A Lithuanian Love Affair,Letters To Sofija,False,6.0,2.0
35340,False,"{'id': 185667, 'name': 'The Complete Monterey ...",0,"[{'id': 10402, 'name': 'Music'}]",NaN,26028,tt0093312,en,Jimi Plays Monterey,It's no exaggeration to say this might be the ...,...,1986-10-23,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Jimi Plays Monterey,Jimi Plays Monterey,False,6.7,7.0
26972,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,79645,tt0067162,en,The Grissom Gang,The Grissom Gang is a remake of the notorious ...,...,1971-05-28,0.0,128.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The psychotic killer, the young heiress...the ...",The Grissom Gang,False,6.9,7.0
15198,False,"{'id': 12077, 'name': ""St. Trinian's Collectio...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 12, 'nam...",NaN,37565,tt1210106,en,St Trinian's 2: The Legend of Fritton's Gold,The girls of St. Trinians are on the hunt for ...,...,2009-12-18,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,St Trinian's 2: The Legend of Fritton's Gold,False,5.4,73.0
6630,False,NaN,118,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.imdb.com/title/tt0078199/,37672,tt0078199,en,"Same Time, Next Year",A man and woman meet by chance at a romantic i...,...,1978-04-23,19.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,They couldn't have celebrated happier annivers...,"Same Time, Next Year",False,6.5,18.0


In [36]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
len(wiki_movies)

0

In [39]:
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie) 
                and 'imdb_link' in movie]
wiki_movies_df=pd.DataFrame(wiki_movies)

In [43]:
wiki_movies_df.columns

Index(['url', 'year', 'imdb_link', 'title', 'Directed by', 'Produced by',
       'Screenplay by', 'Story by', 'Based on', 'Starring', 'Narrated by',
       'Music by', 'Cinematography', 'Edited by', 'Productioncompany ',
       'Distributed by', 'Release date', 'Running time', 'Country', 'Language',
       'Budget', 'Box office', 'Written by', 'Genre', 'Theme music composer',
       'Country of origin', 'Original language(s)', 'Producer(s)', 'Editor(s)',
       'Production company(s)', 'Original network', 'Original release',
       'Productioncompanies ', 'Executive producer(s)',
       'Production location(s)', 'Distributor', 'Picture format',
       'Audio format', 'Voices of', 'Followed by', 'Composer(s)', 'Created by',
       'Also known as', 'Opening theme', 'No. of episodes', 'Preceded by',
       'Adaptation by', 'Suggested by', 'Traditional', 'Mandarin', 'Released',
       'Recorded', 'Venue', 'Length', 'Label', 'Director', 'Producer',
       'Animation by', 'Color process', 'S

In [57]:
wiki_movies_df[wiki_movies_df['Polish'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
7074,https://en.wikipedia.org/wiki/Cold_War_(2018_f...,2018,https://www.imdb.com/title/tt6543652/,Cold War,Paweł Pawlikowski,"[Tanya Seghatchian, Ewa Puszczyńska]",NaN,NaN,NaN,"[Joanna Kulig, Tomasz Kot, Borys Szyc, Agata K...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zimna wojna


In [59]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

In [60]:
clean_movies=[clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']